In [4]:
import requests
import pandas as pd

In [18]:
class OptionChain():
    def __init__(self, symbol='NIFTY', timeout=5) -> None:
        self.__url = "https://www.nseindia.com/api/option-chain-indices?symbol={}".format(symbol)
        self.__session = requests.sessions.Session()
        self.__session.headers = { "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0", "Accept": "*/*", "Accept-Language": "en-US,en;q=0.5" }
        self.__timeout = timeout
        self.__session.get("https://www.nseindia.com/option-chain", timeout=self.__timeout)
    
    def fetch_data(self, expiry_date=None, starting_strike_price=None, number_of_rows=2):
        try:
            data = self.__session.get(url=self.__url, timeout=self.__timeout)
            data = data.json()
            df = pd.json_normalize(data['records']['data'])
            
            if expiry_date != None:
                df = df[(df.expiryDate == expiry_date)]
            
            if starting_strike_price != None:
                df = df[(df.strikePrice >= starting_strike_price)][:number_of_rows]

            # for col in df.columns:
            #      print(col)
            arr_colmn = ['PE.strikePrice','PE.openInterest','PE.changeinOpenInterest','PE.impliedVolatility','CE.openInterest','CE.changeinOpenInterest','CE.impliedVolatility','CE.strikePrice']
            new_df = df[arr_colmn]
            return new_df.head()
        except Exception as ex:
            print('Error: {}'.format(ex))
            self.__session.get("https://www.nseindia.com/option-chain", timeout=self.__timeout)


In [20]:
if __name__ == '__main__':
    oc = OptionChain()
    print(oc.fetch_data(expiry_date='28-Sep-2023'))

    PE.strikePrice  PE.openInterest  PE.changeinOpenInterest  \
1          12000.0           8759.0                   7396.0   
4          13000.0           5812.0                     54.0   
14         14000.0           3131.0                   1710.0   
26         15000.0           2252.0                   -142.0   
36         16000.0          12136.0                    468.0   

    PE.impliedVolatility  CE.openInterest  CE.changeinOpenInterest  \
1                 120.03              4.0                      2.0   
4                 102.67           8256.0                   -599.0   
14                 84.65            190.0                     -3.0   
26                 68.62           1679.0                    -74.0   
36                 55.74           5168.0                   -320.0   

    CE.impliedVolatility  CE.strikePrice  
1                 196.84         12000.0  
4                 147.88         13000.0  
14                134.20         14000.0  
26                 81.